In [1]:
from main import *  # this must go first in order to trigger the computation
from map 

# Feature tables

# pprint(sorted(summary_sleep))
# pprint(sorted(summary_friend))
# pprint(sorted([(collapse_date_ranges(v), k) for k, v in summary_state.iteritems()]))
# pprint(sorted([(collapse_date_ranges(v), k) for k, v in summary_park.iteritems()]))
# pprint(summary_superlative_city)
# pprint(summary_superlative_nature)
# pprint(sorted(summary_animal))
# pprint(summary_got_high)
# pprint(summary_swim)
# pprint(summary_extreme_nsew)
# pprint(summary_facebook)
# pprint(summary_meal)
# pprint(summary_pie)
# pprint(summary_tiki)
# pprint(summary_wedding)
# pprint(sorted([(collapse_date_ranges(v), k) for k, v in summary_cave.iteritems()]))
# pprint(summary_elevation)
# pprint(summary_temperature)
# pprint(summary_other)

m

[(datetime.date(2018, 5, 2), 'Car battery died'),
 (datetime.date(2018, 5, 12), 'O Henry Pun Off'),
 (datetime.date(2018, 5, 27), 'Bumping into Laurel and Andy in Savannah'),
 (datetime.date(2018, 5, 29),
  'After only two days, we are absolutely filthy from the Congaree'),
 (datetime.date(2018, 6, 13),
  'We made silly faces in front of the White House'),
 (datetime.date(2018, 6, 21),
  'Guest sang at a Voices of Gotham barbershop rehearsal'),
 (datetime.date(2018, 7, 8), 'Matt lost his phone in the river :('),
 (datetime.date(2018, 7, 21),
  'Bumping into Jeff and Sharon on the streets of SF'),
 (datetime.date(2018, 7, 27),
  'Matt finished his first journal, started a new one'),
 (datetime.date(2018, 8, 2),
  'We learned the hard way that Google Maps has an option "avoid ferries"'),
 (datetime.date(2018, 8, 2), 'A rock struck the windshield and cracked it')]
